# OHNN: ONE HOT NEURAL NETWORK

We present here One Hot Neural Network

....

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import fede_code.torch_functions as tof
import fede_code.objects as obj
import fede_code.model_trainer as mt
import fede_code.graphics_functions as grafun
from torch.optim import AdamW, RMSprop
from sklearn import svm
from sklearn.metrics import f1_score, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from fede_code.loss_functions import penalizedLoss
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from tabulate import tabulate

## 1. ADULT DATASET

In [22]:
data_path = 'C:\\Users\\drikb\\Desktop\\CRISP\\XAI project\\nn\\data\\adult.csv'

headers = ['age', 'workclass', 'fnlwgt', 'education', 'educational-num', 'marital-status', 'occupation',
           'relationship', 'race', 'gender', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
           'target']

df_adult = pd.read_csv(data_path)

print(df_adult.head())

   age          workclass  fnlwgt   education  educational-num  \
0   39          State-gov   77516   Bachelors               13   
1   50   Self-emp-not-inc   83311   Bachelors               13   
2   38            Private  215646     HS-grad                9   
3   53            Private  234721        11th                7   
4   28            Private  338409   Bachelors               13   

        marital-status          occupation    relationship    race   gender  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital-gain  capital-loss  hours-per-week  native-country  target  
0          2174             0              40   United-S

In [23]:
for col in df_adult.columns:
    if df_adult[col].dtype == 'object':
        df_adult[col] = pd.Categorical(df_adult[col])
        df_adult[col] = df_adult[col].cat.codes
df_adult = df_adult.astype(float)

x = df_adult.drop(columns=['target'])
y = df_adult['target'].values

adult_x_train, adult_x_test, adult_y_train, adult_y_test = train_test_split(x, y, test_size=0.3, random_state=42)

#standardize the dataset
sc = StandardScaler()
sc.fit(adult_x_train)
adult_x_train = sc.transform(adult_x_train)
adult_x_test = sc.transform(adult_x_test)

In [24]:
adult_train_set = mt.TrainDataset(torch.tensor(adult_x_train, dtype=torch.float32),
                     torch.tensor(adult_y_train, dtype=torch.float32))

adult_test_set = mt.TrainDataset(torch.tensor(adult_x_test, dtype=torch.float32),
                           torch.tensor(adult_y_test, dtype=torch.float32))

adult_train_loader = DataLoader(adult_train_set, batch_size=32, shuffle=True)
adult_test_loader = DataLoader(adult_test_set, batch_size=32, shuffle=False)


## 2. PREDICTIVE MODELS

### 2.1 Support Vector Machine (SVM)

In [25]:
svm_clf = svm.SVC(kernel='linear')
svm_clf.fit(adult_x_train,adult_y_train)
predicted_svm = svm_clf.predict(adult_x_test)
f1_0_svm, f1_1_svm = f1_score(adult_y_test,predicted_svm,
                                 average = None,
                                zero_division = 0,
                                labels = np.array([0, 1]))
accuracy_svm = accuracy_score(adult_y_test, predicted_svm)

print('f1 on 0 score SVM:', f1_1_svm)
print('f1 on 1 score SVM:', f1_0_svm)
print('accuracy SVM:', accuracy_svm)

f1 on 0 score SVM: 0.4601661027376192
f1 on 1 score SVM: 0.8922453490513907
accuracy SVM: 0.8203500870099294


In [26]:
# build the dataset of predictions to perform explanation
pred_set_svm = mt.TrainDataset(torch.tensor(adult_x_test, dtype=torch.float32),
                               torch.tensor(predicted_svm, dtype=torch.float32))

pred_loader_svm = DataLoader(pred_set_svm, batch_size=16, shuffle=True)

### 2.2 Deep Neural Network (DNN)

In [27]:
class deepNN(nn.Module):
    def __init__(self, in_features):
        super(deepNN, self).__init__()
        self.linear_1 = nn.Linear(in_features, 128)
        self.act_1 = nn.LeakyReLU()
        self.linear_2 = nn.Linear(128, 64)
        self.act_2 = nn.LeakyReLU()
        self.linear_out = nn.Linear(64, 1)
        self.final_act = nn.Sigmoid()
        self.num_rules = 0 #<--- this allows to use model trainer, must be fixed

    def forward(self, x):
        x = self.linear_1(x)
        x = self.act_1(x)
        x = self.linear_2(x)
        x = self.act_2(x)
        x = self.linear_out(x)
        x = self.final_act(x)
        return x

In [28]:
dnn = deepNN(in_features = 14)

loss_fn = penalizedLoss(loss_fn = nn.BCELoss(reduction='sum'), #<--- reduction='sum' is mandatory
                        parameters = dnn.parameters(),
                        l1_lambda = 0.5,
                        l2_lambda = 0.5)

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('GPU')
else:
    device = torch.device('cpu')
    print('CPU')


trainer = mt.modelTrainer(model=dnn,
                          loss_fn=loss_fn,
                          optimizer=AdamW)

trainer.train_model(train_data_loader=adult_train_loader,
                    num_epochs=100,
                    device=device,
                    learning_rate=1e-5,
                    display_log='epoch',
                   store_weights_history=False,
                   store_weights_grad=False)


CPU


  0%|          | 0/100 [00:00<?, ?it/s]

In [29]:
loss_dnn, accuracy_dnn, f1_0_dnn, f1_1_dnn = trainer.eval_model(adult_test_loader, device=device)
print(f'Average loss: {loss_dnn}')
print(f'Accuracy: {accuracy_dnn}')
print(f'F1 score on label 0: {f1_0_dnn}')
print(f'F1 score on label 1: {f1_1_dnn}')

  0%|          | 0/306 [00:00<?, ?it/s]

Average loss: 0.32505376195697705
Accuracy: 0.8490121813901116
F1 score on label 0: 0.9021680135965053
F1 score on label 1: 0.6297476101422543


In [30]:
predicted_dnn = trainer.predict(adult_test_loader, device=device, prob=False)

  0%|          | 0/306 [00:00<?, ?it/s]

In [31]:
# build the dataset of predictions to perform explanation
pred_set_dnn = mt.TrainDataset(torch.tensor(adult_x_test, dtype=torch.float32),
                               torch.tensor(predicted_dnn.detach().numpy(), dtype=torch.float32))

pred_loader_dnn = DataLoader(pred_set_dnn, batch_size=16, shuffle=True)

## 3. EXPLANATION MODELS

### 3.1 LOGISTIC REGRESSION (LR)

In [32]:
log_reg = LogisticRegression()

log_reg_svm = log_reg.fit(adult_x_test, predicted_svm)
log_reg_dnn = log_reg.fit(adult_x_test, predicted_dnn)

log_reg_pred_svm = log_reg_svm.predict(adult_x_test)
log_reg_pred_dnn = log_reg_dnn.predict(adult_x_test)

In [33]:
# SVM
log_reg_accuracy_svm = (log_reg_pred_svm == predicted_svm).sum()/len(predicted_svm)
log_reg_f1_0_svm, log_reg_f1_1_svm = f1_score(predicted_svm,log_reg_pred_svm,
                                              average = None,
                                              zero_division = 0,
                                              labels = np.array([0, 1]))

# DNN
log_reg_accuracy_dnn = (log_reg_pred_dnn == predicted_dnn.detach().numpy()).sum()/len(predicted_dnn)
log_reg_f1_0_dnn, log_reg_f1_1_dnn = f1_score(predicted_dnn,log_reg_pred_dnn,
                                              average = None,
                                              zero_division = 0,
                                              labels = np.array([0, 1]))

print('Explanation accuracy on SVM:', log_reg_accuracy_svm)
print('Explanation f1 of 0 on SVM:', log_reg_f1_0_svm)
print('Explanation f1 of 1 on SVM:', log_reg_f1_1_svm)

print('Explanation accuracy on DNN:', log_reg_accuracy_dnn)
print('Explanation f1 of 0 on DNN:', log_reg_f1_0_dnn)
print('Explanation f1 of 1 on DNN:', log_reg_f1_1_dnn)

Explanation accuracy on SVM: 0.9215886989456444
Explanation f1 of 0 on SVM: 0.954861520329994
Explanation f1 of 1 on SVM: 0.7017133956386293
Explanation accuracy on DNN: 0.921179240454499
Explanation f1 of 0 on DNN: 0.9519890260631001
Explanation f1 of 1 on DNN: 0.78


### 3.2 DECISION TREE (DT)

In [34]:
dt = DecisionTreeClassifier(criterion='gini',
                            max_depth = 4,
                            max_leaf_nodes = 16)

dt_svm = dt.fit(adult_x_test, predicted_svm)
dt_dnn = dt.fit(adult_x_test, predicted_dnn)

dt_pred_svm = dt_svm.predict(adult_x_test)
dt_pred_dnn = dt_dnn.predict(adult_x_test)

In [35]:
# SVM
dt_accuracy_svm = (dt_pred_svm == predicted_svm).sum()/len(predicted_svm)
dt_f1_0_svm, dt_f1_1_svm = f1_score(predicted_svm,dt_pred_svm,
                                              average = None,
                                              zero_division = 0,
                                              labels = np.array([0, 1]))

# DNN
dt_accuracy_dnn = (dt_pred_dnn == predicted_dnn.detach().numpy()).sum()/len(predicted_dnn)
dt_f1_0_dnn, dt_f1_1_dnn = f1_score(predicted_dnn,dt_pred_dnn,
                                              average = None,
                                              zero_division = 0,
                                              labels = np.array([0, 1]))

print('Explanation accuracy on SVM:', dt_accuracy_svm)
print('Explanation f1 of 0 on SVM:', dt_f1_0_svm)
print('Explanation f1 of 1 on SVM:', dt_f1_1_svm)

print('Explanation accuracy on DNN:', dt_accuracy_dnn)
print('Explanation f1 of 0 on DNN:', dt_f1_0_dnn)
print('Explanation f1 of 1 on DNN:', dt_f1_1_dnn)

Explanation accuracy on SVM: 0.9068481932644078
Explanation f1 of 0 on SVM: 0.9465711601690934
Explanation f1 of 1 on SVM: 0.6368715083798884
Explanation accuracy on DNN: 0.9404237895383355
Explanation f1 of 0 on DNN: 0.9638509316770186
Explanation f1 of 1 on DNN: 0.8307155322862129


### ONE HOT NEURAL NETWORK (OHNN)

In [37]:
ohnn_svm = obj.oneHotRuleNN(input_size=14,
                        num_neurons=4, #tree max depth
                        num_rules=16, #tree max leaves
                        final_activation=nn.Sigmoid,
                        rule_weight_constraint=tof.getMax,
                        out_weight_constraint=None,
                        rule_bias=True,
                        neuron_bias=False,
                        rule_activation=nn.Sigmoid,
                        neuron_activation=None,
                        out_bias=False,
                        force_positive_hidden_init=False,
                        force_positive_out_init=False,
                        dtype=torch.float32)

#loss_fn = nn.BCELoss(reduction='sum')
loss_fn = penalizedLoss(loss_fn = nn.BCELoss(reduction='sum'), #<--- reduction='sum' is mandatory
                        parameters = ohnn_svm.rule_weight.linear.weight,
                        l1_lambda = 0.3,
                        l2_lambda = 0.0)

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('GPU')
else:
    device = torch.device('cpu')
    print('CPU')


trainer_svm = mt.modelTrainer(model=ohnn_svm,
                          loss_fn=loss_fn,
                          optimizer=AdamW)

trainer_svm.train_model(train_data_loader=pred_loader_svm,
                    num_epochs=100,
                    device=device,
                    learning_rate=1e-3,
                    display_log='epoch')


CPU


  0%|          | 0/100 [00:00<?, ?it/s]

In [39]:
ohnn_dnn = obj.oneHotRuleNN(input_size=14,
                        num_neurons=4, #tree max depth
                        num_rules=16, #tree max leaves
                        final_activation=nn.Sigmoid,
                        rule_weight_constraint=tof.getMax,
                        out_weight_constraint=None,
                        rule_bias=True,
                        neuron_bias=False,
                        rule_activation=nn.Sigmoid,
                        neuron_activation=None,
                        out_bias=False,
                        force_positive_hidden_init=False,
                        force_positive_out_init=False,
                        dtype=torch.float32)

#loss_fn = nn.BCELoss(reduction='sum')
loss_fn = penalizedLoss(loss_fn = nn.BCELoss(reduction='sum'), #<--- reduction='sum' is mandatory
                        parameters = ohnn_dnn.rule_weight.linear.weight,
                        l1_lambda = 0.3,
                        l2_lambda = 0.0)

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('GPU')
else:
    device = torch.device('cpu')
    print('CPU')


trainer_dnn = mt.modelTrainer(model=ohnn_dnn,
                          loss_fn=loss_fn,
                          optimizer=AdamW)

trainer_dnn.train_model(train_data_loader=pred_loader_dnn,
                    num_epochs=100,
                    device=device,
                    learning_rate=1e-3,
                    display_log='epoch')

CPU


  0%|          | 0/100 [00:00<?, ?it/s]

In [40]:
ohnn_loss_svm, ohnn_accuracy_svm, ohnn_f1_0_svm, ohnn_f1_1_svm = trainer_svm.eval_model(pred_loader_svm, device='cpu')
ohnn_loss_dnn, ohnn_accuracy_dnn, ohnn_f1_0_dnn, ohnn_f1_1_dnn = trainer_dnn.eval_model(pred_loader_dnn, device='cpu')

print('Explanation accuracy on SVM:', ohnn_accuracy_svm)
print('Explanation f1 of 0 on SVM:', ohnn_f1_0_svm)
print('Explanation f1 of 1 on SVM:', ohnn_f1_1_svm)

print('Explanation accuracy on DNN:', ohnn_accuracy_dnn)
print('Explanation f1 of 0 on DNN:', ohnn_f1_0_dnn)
print('Explanation f1 of 1 on DNN:', ohnn_f1_1_dnn)

  0%|          | 0/611 [00:00<?, ?it/s]

  0%|          | 0/611 [00:00<?, ?it/s]

Explanation accuracy on SVM: 0.9766608660047088
Explanation f1 of 0 on SVM: 0.9867155930149829
Explanation f1 of 1 on SVM: 0.6494375600758586
Explanation accuracy on DNN: 0.9399119664244038
Explanation f1 of 0 on DNN: 0.9622440426442823
Explanation f1 of 1 on DNN: 0.7596399963012087


## 4. RESULTS

In [41]:
scores = {
    'Logistic regression': {
        'SVM': {'Accuracy': log_reg_accuracy_svm, 'F1 Score (Class 0)': log_reg_f1_0_svm, 'F1 Score (Class 1)': log_reg_f1_1_svm},
        'DNN': {'Accuracy': log_reg_accuracy_dnn, 'F1 Score (Class 0)': log_reg_f1_0_dnn, 'F1 Score (Class 1)': log_reg_f1_1_dnn},
    },
    'Decision tree': {
        'SVM': {'Accuracy': dt_accuracy_svm, 'F1 Score (Class 0)': dt_f1_0_svm, 'F1 Score (Class 1)': dt_f1_1_svm},
        'DNN': {'Accuracy': dt_accuracy_dnn, 'F1 Score (Class 0)': dt_f1_0_dnn, 'F1 Score (Class 1)': dt_f1_1_dnn},
    },
    'OHNN': {
        'SVM': {'Accuracy': ohnn_accuracy_svm, 'F1 Score (Class 0)': ohnn_f1_0_svm, 'F1 Score (Class 1)': ohnn_f1_1_svm},
        'DNN': {'Accuracy': ohnn_accuracy_dnn, 'F1 Score (Class 0)': ohnn_f1_0_dnn, 'F1 Score (Class 1)': ohnn_f1_1_dnn},
    }
}

In [42]:
df_scores = pd.DataFrame.from_dict({(i, j): scores[i][j] 
                              for i in scores.keys() 
                              for j in scores[i].keys()}, orient='index')

# Rename index and columns
df_scores.index.names = ['Model', 'Explained Predictions']
df_scores.reset_index(inplace=True)

# Format the DataFrame to round the values to a certain number of decimal places (e.g., 2)
df_scores = df_scores.round(4)

# Print the table using tabulate
table = tabulate(df_scores, headers='keys', tablefmt='pretty', showindex=False)

print(table)

+---------------------+-----------------------+----------+--------------------+--------------------+
|        Model        | Explained Predictions | Accuracy | F1 Score (Class 0) | F1 Score (Class 1) |
+---------------------+-----------------------+----------+--------------------+--------------------+
| Logistic regression |          SVM          |  0.9216  |       0.9549       |       0.7017       |
| Logistic regression |          DNN          |  0.9212  |       0.952        |        0.78        |
|    Decision tree    |          SVM          |  0.9068  |       0.9466       |       0.6369       |
|    Decision tree    |          DNN          |  0.9404  |       0.9639       |       0.8307       |
|        OHNN         |          SVM          |  0.9767  |       0.9867       |       0.6494       |
|        OHNN         |          DNN          |  0.9399  |       0.9622       |       0.7596       |
+---------------------+-----------------------+----------+--------------------+------------